# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120994e+02     1.687180e+00
 * time: 1.1469910144805908
     1     1.162566e+01     1.252317e+00
 * time: 3.457595109939575
     2    -1.239379e+01     1.263519e+00
 * time: 4.1448609828948975
     3    -3.422833e+01     9.171553e-01
 * time: 5.182893991470337
     4    -4.798478e+01     5.770044e-01
 * time: 6.168481111526489
     5    -5.699412e+01     1.739505e-01
 * time: 7.137538909912109
     6    -5.978979e+01     9.959664e-02
 * time: 7.794487953186035
     7    -6.081873e+01     6.043434e-02
 * time: 8.444252014160156
     8    -6.123410e+01     6.749937e-02
 * time: 9.081609010696411
     9    -6.157831e+01     4.152623e-02
 * time: 9.725692987442017
    10    -6.180597e+01     3.319845e-02
 * time: 10.384742021560669
    11    -6.197513e+01     2.358813e-02
 * time: 11.059993028640747
    12    -6.204779e+01     1.833292e-02
 * time: 11.719477891921997
    13    -6.211701e+01     1.375369e-02
 * time: 12.384156942367554
 

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671058
    AtomicLocal         -18.8557682
    AtomicNonlocal      14.8522661
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666460628
